ElGamal
===============

## Diffie-Hellman key distribution scheme

IN 1975, Diffie and Hellman introduced the concept of public key cryptography.

Suppose that $A$ and $B$ want to share a secret $K_{AB}$ , where A has a secret $x_A$ and $B$ has a secret $x_B$ . Let $p$ be a large prime and $\alpha$ be a **primitive element** mod $p$, both known. computes $y \equiv \alpha^{x_a} \mod p$ , and sends $y_A$ . Similarly, $B$ computes $y_B \equiv \alpha^{x_B} \mod p$ and sends $y_B$ . Then the secret $K_{AB}$ is computed as:

\begin{align}
K_{AB} &\equiv \alpha^{x_ax_B} \mod p\\
&\equiv y_A^{x_B} \mod p\\
&\equiv y_B^{x_A} \mod p
\end{align}

In any of the cryptographic systems based on discrete logarithms, p must be chosen such that $p - 1$ has at least one large prime factor. If $p - 1$ has only small prime factors, then computing discrete logarithms is easy

In [1]:
from klefki.numbers.primes import generate_prime
from klefki.algebra.meta import field
from klefki.algebra.utils import randfield
from math import gcd

In [2]:
p = generate_prime(32)
F = field(p)
mF = field(p-1)
G = F(generate_prime(32))

In [3]:
xa, xb = randfield(F).value, randfield(F).value
ya, yb = G ** xa, G ** xb
G ** (xa * xb) == ya ** xb == yb ** xa

True

## ElGamal signature scheme

Let m be a document to be signed, where $0 \le m \ge p - 1$. The public file still consists of the public key $y \equiv x \mod p$ for each user.

To sign a document, a user $A$ should be able to use the secret key $x_A$ to find a signature for $m$ in such a way that all users can verify the authenticity of the signature by using the public key $y_A$ (together with $\alpha$ and $p$), and no one can forge a signature without knowing the secret $x_A$ .

The signature for $m$ is the pair $(r, s), 0 \le r, s < p - 1$, chosen such that the equation

$$
\alpha^m \equiv y^r r^s \mod p
$$

is satisfied.

In [4]:
m = randfield(F).value
alpha = G
x = xa
y = ya

* The Signing Procedure

The signing procedure consists of the following three steps.

1) Choose a random number $k$, uniformly between $0$ and $p - 1$, such that $gcd(k, p - 1) = 1$.

2) Compute

$$
r \equiv \alpha^k \mod p\\
\alpha^m \equiv \alpha^{xr}\alpha^{ks} \mod p
$$

which can be solved for $s$ by using

$$
m \equiv xr + ks \mod (p-1)
$$

Above Equation as a solution for $s$ if $k$ is chosen such that $gcd(k, p - 1) = 1$.

In [5]:
k = generate_prime(32)
assert gcd(k, p-1) == 1

In [6]:
from klefki.numbers import invmod
r = (alpha ** k).value
s = ((mF(m) - mF(r) * mF(x)) * ~mF(k)).value


In [7]:
assert pow(alpha, m) ==  pow(alpha, (x * r + k * s))

* The Verification Procedure

Given $m, r$, an $s$, it is easy to verify the authenticity of the signature by computing both sides of $\alpha^m \equiv y^r r^s \mod p$ and checking that they are equal.

In [8]:
alpha ** m == y ** r * F(r) ** s

True

## ElGamal encryption over Cyclic Group

In [9]:
from klefki.curves.secp256k1 import EllipticCurveGroupSecp256k1 as Curve
from klefki.curves.secp256k1 import FiniteFieldSecp256k1 as F

from klefki.algebra.meta import field

### Key generation

The first party, Alice, generates a key pair as follows:

* Generate an efficient description of a cyclic group $G$, of order $q$  with generator $g$. Let $e$ represent the unit element of $G$.

* Choose an integer $x\leftarrow Z_q$

* Compute $h:=g^x$

* he public key consists of the values $( G , q , g , h )$. Alice publishes this public key and retains $x$ as her private key, which must be kept secret.

In [10]:
G = Curve.G
sk = randfield(F)
H = G @ sk


### Encryption

A second party, Bob, encrypts a message $M$ to Alice under her public key $( G , q , g , h )$  as follows: 

* Map the message $M$ to an element $m$ of $G$ using a reversible mapping function.

* Choose an integer $y$ randomly from $\{ 1 , … , q − 1 \}$ 

* Compute $s := h^y$. This is called the shared secret.

* Compute $c_1 := g^y$

* Compute $c_2 := m \cdot s$

Bob sends the ciphertext $( c_1 , c_2 )$ to Alice.

$$
c=g^r, h^{mr}
$$

In [11]:
M = F(1235)

In [12]:
y = randfield(F).value
m = Curve.lift_x(M)

c1 = G @ y
c2 = H @ y + m

(c1, c2)

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::0x2bdccd6af7b6d7cf90bfeaa8fa922a4069162ce391f4ad5f9257bb07e2535ca6, FiniteFieldSecp256k1::0x1a90f2fbf9c51a5f0f478ca1bf9afa916b294654c8a73df64be31c8a90d234f5),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::0xef75ef7bd44f017acb03cb9f6327534363d878a7343c84b2475ce461b6707c9d, FiniteFieldSecp256k1::0x1fd86243e16fb44649eedb19a29279f410a4f5887c60ce9a5a02179b3644abeb))

### Decryption

Alice decrypts a ciphertext $c_1, c_2$ with her private key $sk$ as follows: 

* Compute $s := c_1^{x}$.

* Compute $s^{-1}$,  the inverse of $s$ in the group $G$.

* Compute $m:=c_2\cdot s^{-1}$



\begin{align}
m&=\frac{c_2}{c_1^x}=\frac{H^{rm}}{G^{xr}}=\frac{G^{xrm}}{G^{xr}}
\end{align}


In [13]:
m = (c2 - (c1 @ sk))

In [14]:
m.x == M

True

# Test


In [15]:
from klefki.curves.secp256k1 import EllipticCurveGroupSecp256k1 as Curve
from klefki.curves.secp256k1 import FiniteFieldCyclicSecp256k1 as CF
from klefki.curves.secp256k1 import FiniteFieldSecp256k1 as F
from klefki.algebra.meta import field
from klefki.blockchain.ethereum.private import decode_privkey, encode_privkey
from klefki.crypto.ecdsa.secp256k1 import pubkey


from klefki.utils import int_to_byte, byte_to_int

key = decode_privkey("65860affb4b570dba06db294aa7c676f68e04a5bf2721243ad3cbc05a79c68c0")

In [16]:

pubkey = G @ F(key)

In [17]:
[int(i) for i in int_to_byte(pubkey.x.value)]

[226,
 15,
 49,
 60,
 133,
 119,
 254,
 51,
 180,
 4,
 209,
 133,
 17,
 253,
 134,
 129,
 149,
 245,
 53,
 173,
 45,
 62,
 36,
 113,
 168,
 153,
 24,
 91,
 137,
 141,
 81,
 47]

In [18]:
m_x = [253, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 116, 101, 115, 116]
m_y = [102, 240, 6, 242, 139, 13, 202, 191, 92, 132, 250, 2, 205, 187, 196, 131, 154, 248, 95, 123, 242, 215, 83, 237, 38, 195, 221, 29, 137, 141, 2, 132]

In [19]:
m_x = F(byte_to_int(m_x))
m_y = F(byte_to_int(m_y))

In [20]:
m = Curve(m_x, m_y)
m

EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::0xfd00000000000000000000000000000000000000000000000000000074657374, FiniteFieldSecp256k1::0x66f006f28b0dcabf5c84fa02cdbbc4839af85f7bf2d753ed26c3dd1d898d0284)

In [21]:
r = F(0x1f9275dbafdfba81942eb3330b07f38cbee4ebb86bdc2174af9648d5f5509a54)

In [22]:
[int(i) for i in int_to_byte(r.value)]

[31,
 146,
 117,
 219,
 175,
 223,
 186,
 129,
 148,
 46,
 179,
 51,
 11,
 7,
 243,
 140,
 190,
 228,
 235,
 184,
 107,
 220,
 33,
 116,
 175,
 150,
 72,
 213,
 245,
 80,
 154,
 84]

In [23]:
c1 = G @ r


In [24]:
c1

EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::0xfca855e9dc774cd9346ca71beabcc55f48d594d46fff063b09866f79af09bd69, FiniteFieldSecp256k1::0x142d0d3df53288b7b6d2a97854cc4d8a0c74320973628af5183ddf9037b4e73b)

In [25]:
[int(i) for i in int_to_byte(c1.x.value)]

[252,
 168,
 85,
 233,
 220,
 119,
 76,
 217,
 52,
 108,
 167,
 27,
 234,
 188,
 197,
 95,
 72,
 213,
 148,
 212,
 111,
 255,
 6,
 59,
 9,
 134,
 111,
 121,
 175,
 9,
 189,
 105]

In [26]:
[int(i) for i in int_to_byte(c1.y.value)]

[20,
 45,
 13,
 61,
 245,
 50,
 136,
 183,
 182,
 210,
 169,
 120,
 84,
 204,
 77,
 138,
 12,
 116,
 50,
 9,
 115,
 98,
 138,
 245,
 24,
 61,
 223,
 144,
 55,
 180,
 231,
 59]

In [27]:
ss = pubkey @ r

In [28]:
ss.x.value

98638154376543494645275106847376508464438552902924450790959126183159388906274

In [29]:
[int(i) for i in int_to_byte(ss.x.value)]

[218,
 19,
 55,
 137,
 15,
 46,
 160,
 160,
 208,
 222,
 206,
 77,
 46,
 79,
 32,
 80,
 64,
 243,
 93,
 23,
 223,
 130,
 148,
 226,
 131,
 17,
 254,
 95,
 43,
 95,
 35,
 34]

In [30]:
[int(i) for i in int_to_byte(ss.y.value)]

[106,
 127,
 47,
 58,
 214,
 6,
 110,
 28,
 171,
 176,
 73,
 11,
 34,
 28,
 125,
 10,
 82,
 154,
 84,
 154,
 11,
 80,
 191,
 68,
 111,
 197,
 98,
 224,
 84,
 116,
 208,
 115]

In [31]:
c2 = ss + m


In [32]:
t = c1 @ key

In [33]:
[int(i) for i in int_to_byte(t.y.value)]

[106,
 127,
 47,
 58,
 214,
 6,
 110,
 28,
 171,
 176,
 73,
 11,
 34,
 28,
 125,
 10,
 82,
 154,
 84,
 154,
 11,
 80,
 191,
 68,
 111,
 197,
 98,
 224,
 84,
 116,
 208,
 115]

In [34]:
d = (c2 - (c1 @ key))

In [35]:
d.x == m.x

True

## Ref:
    
   * T. ElGamal. A Public Key Cryptosystem and a Signature Scheme Based on Discrete Logarithms. IEEE Trans. Info. Theory, IT 31:469–472, 1985.
    
   * ElGamal encryption https://en.wikipedia.org/wiki/ElGamal_encryption
    
   * http://www.docsdrive.com/pdfs/ansinet/itj/2005/299-306.pdf